In [13]:
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from tqdm.notebook import tqdm
import contextily as cx
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point

tqdm.pandas()

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

In [14]:
cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows={}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory

In [15]:
start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]
time_ranges

[datetime.datetime(2023, 6, 1, 0, 0),
 datetime.datetime(2023, 6, 1, 0, 1),
 datetime.datetime(2023, 6, 1, 0, 2),
 datetime.datetime(2023, 6, 1, 0, 3),
 datetime.datetime(2023, 6, 1, 0, 4),
 datetime.datetime(2023, 6, 1, 0, 5),
 datetime.datetime(2023, 6, 1, 0, 6),
 datetime.datetime(2023, 6, 1, 0, 7),
 datetime.datetime(2023, 6, 1, 0, 8),
 datetime.datetime(2023, 6, 1, 0, 9),
 datetime.datetime(2023, 6, 1, 0, 10),
 datetime.datetime(2023, 6, 1, 0, 11),
 datetime.datetime(2023, 6, 1, 0, 12),
 datetime.datetime(2023, 6, 1, 0, 13),
 datetime.datetime(2023, 6, 1, 0, 14),
 datetime.datetime(2023, 6, 1, 0, 15),
 datetime.datetime(2023, 6, 1, 0, 16),
 datetime.datetime(2023, 6, 1, 0, 17),
 datetime.datetime(2023, 6, 1, 0, 18),
 datetime.datetime(2023, 6, 1, 0, 19),
 datetime.datetime(2023, 6, 1, 0, 20),
 datetime.datetime(2023, 6, 1, 0, 21),
 datetime.datetime(2023, 6, 1, 0, 22),
 datetime.datetime(2023, 6, 1, 0, 23),
 datetime.datetime(2023, 6, 1, 0, 24),
 datetime.datetime(2023, 6, 1, 0, 2

In [6]:
traj = rows[(9112856,)]
traj.timestamps()

[datetime.datetime(2023, 6, 1, 0, 0, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 4, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 6, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 10, 42, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 14, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 16, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 22, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 37, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 40, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 43, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 53, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 1, 8, 10, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 1, 14, 2, tzinfo=tzoffset(None, 7200)),
 datetime.dat

In [7]:
traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= timedelta(minutes=1))
traj_resampled.timestamps()

[datetime.datetime(2023, 6, 1, 0, 0, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 1, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 2, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 3, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 4, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 5, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 6, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 7, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 8, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 9, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 10, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 11, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 12, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 13, tzinfo=tzoffset(None, 7200)),
 datetime.datetime(2023, 6, 1, 0, 14, tzinfo=tzoffset(None

In [16]:
start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
print(f"Start time: {start_time}")
print(f"End time: {end_time}")


Start time: 2023-06-01 00:00:00
End time: 2023-06-01 23:57:00


In [14]:
print(len(time_ranges))
print(len(traj_resampled.timestamps()))

1440
1438


In [18]:
start_index = time_ranges.index(start_time)
end_index = time_ranges.index(end_time)
print(start_index)
print(end_index)

0
1437


In [27]:

empty_point_wkt = Point().wkt  # "POINT EMPTY"

matrix = np.full(1440, empty_point_wkt, dtype=object)



In [30]:

values= traj_resampled.values()
for i in range(start_index, end_index + 1):
    matrix[i] = values[i].wkt

In [31]:
matrix

array(['POINT (12.3227 56.1102)', 'POINT (12.3227 56.1102)',
       'POINT (12.322700000165371 56.11018999999999)', ...,
       'POINT (12.322700001519749 56.110176086956514)', 'POINT EMPTY',
       'POINT EMPTY'], dtype=object)

# Creating the matrix

In [9]:
%%time 

import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from tqdm.notebook import tqdm
import contextily as cx
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point

tqdm.pandas()

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows={}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory


CPU times: user 10.1 s, sys: 584 ms, total: 10.7 s
Wall time: 15.7 s


In [10]:
%%time

from tqdm.notebook import tqdm
start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]

mmsi_index = {}
empty_point_wkt = Point().wkt  # "POINT EMPTY"
#Create a numpy matrix of size 1440x len(mmsi_list) with empty points
matrix = np.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)


        


CPU times: user 222 ms, sys: 85.2 ms, total: 307 ms
Wall time: 316 ms


In [22]:
mmsi_list[7]

(109080281,)

In [23]:
traj = rows[mmsi_list[8]]

In [24]:
traj.num_instants()

2954

In [26]:
traj = traj.temporal_precision(timedelta(minutes=1))

In [49]:
start = traj.start_timestamp()
start            

datetime.datetime(2023, 6, 1, 15, 27, tzinfo=tzoffset(None, 7200))

In [39]:
traj_resampled = traj.temporal_sample(start=time_ranges[0],duration= timedelta(minutes=1))
traj_resampled

TGeogPointSeq({POINT(12.118724999943645 55.591717307899245)@2023-06-01 15:27:00+02, POINT(12.097305833308912 55.584297500089676)@2023-06-01 15:28:00+02, POINT(12.076010833333333 55.5538075)@2023-06-01 15:29:00+02, POINT(12.086706666666666 55.51652833333333)@2023-06-01 15:30:00+02, POINT(12.091881666666668 55.48168666666667)@2023-06-01 15:31:00+02, POINT(12.096208333941222 55.447913333357796)@2023-06-01 15:32:00+02, POINT(12.10752500091183 55.411915000024464)@2023-06-01 15:33:00+02, POINT(12.121036666666667 55.37485)@2023-06-01 15:34:00+02, POINT(12.130360833333333 55.33860583333333)@2023-06-01 15:35:00+02, POINT(12.175428333333333 55.32172833333333)@2023-06-01 15:36:00+02, POINT(12.237058333333334 55.316298333333336)@2023-06-01 15:37:00+02, POINT(12.29725 55.31845333333333)@2023-06-01 15:38:00+02, POINT(12.354156666666666 55.32509)@2023-06-01 15:39:00+02, POINT(12.410958331742918 55.335106667266736)@2023-06-01 15:40:00+02, POINT(12.45741082856209 55.34840333453347)@2023-06-01 15:41:00+

In [51]:
start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
start_index = time_ranges.index(start_time)
end_index = time_ranges.index(end_time)

In [42]:
len(traj_resampled.timestamps())

129

In [44]:
import numpy as np

trajectory_array = np.array([point.wkt for point in traj_resampled.values()])

In [45]:
trajectory_array

array(['POINT (12.118724999943645 55.591717307899245)',
       'POINT (12.097305833308912 55.584297500089676)',
       'POINT (12.076010833333333 55.5538075)',
       'POINT (12.086706666666666 55.51652833333333)',
       'POINT (12.091881666666668 55.48168666666667)',
       'POINT (12.096208333941222 55.447913333357796)',
       'POINT (12.10752500091183 55.411915000024464)',
       'POINT (12.121036666666667 55.37485)',
       'POINT (12.130360833333333 55.33860583333333)',
       'POINT (12.175428333333333 55.32172833333333)',
       'POINT (12.237058333333334 55.316298333333336)',
       'POINT (12.29725 55.31845333333333)',
       'POINT (12.354156666666666 55.32509)',
       'POINT (12.410958331742918 55.335106667266736)',
       'POINT (12.45741082856209 55.34840333453347)',
       'POINT (12.443135 55.332586666666664)',
       'POINT (12.429321666666667 55.298235)',
       'POINT (12.41095666695198 55.26794666686348)',
       'POINT (12.364533333618649 55.271563333530146)',
  

In [55]:
matrix[0,start_index:end_index]

array(['POINT (12.118724999943645 55.591717307899245)',
       'POINT (12.097305833308912 55.584297500089676)',
       'POINT (12.076010833333333 55.5538075)',
       'POINT (12.086706666666666 55.51652833333333)',
       'POINT (12.091881666666668 55.48168666666667)',
       'POINT (12.096208333941222 55.447913333357796)',
       'POINT (12.10752500091183 55.411915000024464)',
       'POINT (12.121036666666667 55.37485)',
       'POINT (12.130360833333333 55.33860583333333)',
       'POINT (12.175428333333333 55.32172833333333)',
       'POINT (12.237058333333334 55.316298333333336)',
       'POINT (12.29725 55.31845333333333)',
       'POINT (12.354156666666666 55.32509)',
       'POINT (12.410958331742918 55.335106667266736)',
       'POINT (12.45741082856209 55.34840333453347)',
       'POINT (12.443135 55.332586666666664)',
       'POINT (12.429321666666667 55.298235)',
       'POINT (12.41095666695198 55.26794666686348)',
       'POINT (12.364533333618649 55.271563333530146)',
  

In [52]:
start_index 

927

In [53]:

matrix[0, start_index:end_index+1] = trajectory_array

In [ ]:
for j in range(start_index, end_index ):
    # update the matrix with the point
    matrix[i][j] = values[j-start_index].wkt

In [ ]:
for i in tqdm(range(len(mmsi_list))):
    try:
        mmsi_index[mmsi_list[i]] = i
        traj = rows[mmsi_list[i]]
        if traj.num_instants() > 3:
            traj = traj.temporal_precision(timedelta(minutes=1))
            start = traj.start_timestamp()
            traj_resampled = traj.temporal_sample(start=start,duration= timedelta(minutes=1))
            #traj_resampled.timestamps()
            start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
            end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
            # print(f"Start time: {start_time}")
            # print(f"End time: {end_time}")
            start_index = time_ranges.index(start_time)
            end_index = time_ranges.index(end_time)
            # print(start_index)
            # print(end_index)
            values= traj_resampled.values()

            for j in range(start_index, end_index ):
                # update the matrix with the point
                matrix[i][j] = values[j-start_index].wkt
    except Exception as e:
        print(f"Error: {e} for MMSI: {mmsi_list[i]}")

In [135]:
matrix

array([['POINT (12.3227 56.1102)',
        'POINT (12.322700000082685 56.1101985)',
        'POINT (12.322700000440992 56.110173333333336)', ...,
        'POINT EMPTY', 'POINT EMPTY', 'POINT EMPTY'],
       ['POINT EMPTY', 'POINT EMPTY', 'POINT EMPTY', ...,
        'POINT (10.5245 54.85059997395833)', 'POINT EMPTY',
        'POINT EMPTY']], dtype=object)

In [137]:
for mmsi in mmsi_list:
    i = mmsi_index[mmsi]
    t_index = timestamps.index(datetime(2023, 6, 1, 3, 0, 0))
    print(f"Time: {timestamps[t_index]}")
    print(f"Point: {matrix[i][t_index]}")

Time: 2023-06-01 03:00:00
Point: POINT (12.322672455089819 56.11010000000384)
Time: 2023-06-01 03:00:00
Point: POINT (10.8616 55.0625)


In [146]:
from pympler import asizeof
size_in_bytes = asizeof.asizeof(matrix)
size_in_megabytes = size_in_bytes / (1024 * 1024)
print(f"Size of the matrix: {size_in_megabytes} MB")

Size of the matrix: 63.95155334472656 MB


In [151]:
# dimension of matrix
matrix.shape

#Get columns 0
ts_1 =matrix[:, 0]
ts_1

array(['POINT EMPTY', 'POINT (8.42333 55.4718)', 'POINT EMPTY', ...,
       'POINT EMPTY', 'POINT EMPTY', 'POINT EMPTY'], dtype=object)

In [154]:
# Create a list from ts_1 with all the points that are not empty
# using numpy operations
points = ts_1[ts_1 != empty_point_wkt]
points

array(['POINT (8.42333 55.4718)', 'POINT (12.3227 56.1102)',
       'POINT (8.565365280701755 55.08710000000018)', ...,
       'POINT (8.989036848640897 53.85318377997052)',
       'POINT (7.112356 53.33180000047196)', 'POINT (10.619 55.0587)'],
      dtype=object)

# Fully working example

In [ ]:

import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point

pymeos_initialize()

host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows={}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory

from tqdm import tqdm
start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]

mmsi_index = {}
empty_point_wkt = Point().wkt  # "POINT EMPTY"
#Create a numpy matrix of size 1440x len(mmsi_list) with empty points
matrix = np.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)

for i in tqdm(range(len(mmsi_list))):
    try:
        mmsi_index[mmsi_list[i]] = i
        traj = rows[mmsi_list[i]]
        if traj.num_instants() > 3:
            traj = traj.temporal_precision(timedelta(minutes=1))
            start = traj.start_timestamp()
            traj_resampled = traj.temporal_sample(start=start,duration= timedelta(minutes=1))
            #traj_resampled.timestamps()
            start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
            end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
            # print(f"Start time: {start_time}")
            # print(f"End time: {end_time}")
            start_index = time_ranges.index(start_time)
            end_index = time_ranges.index(end_time)
            # print(start_index)
            # print(end_index)
            values= traj_resampled.values()

            for j in range(start_index, end_index ):
                # update the matrix with the point
                matrix[i][j] = values[j-start_index].wkt
    except Exception as e:
        print(f"Error: {e} for MMSI: {mmsi_list[i]}")
        


vlayer = QgsVectorLayer("Point", "MobilityBD Data", "memory")
pr = vlayer.dataProvider()
pr.addAttributes([QgsField("time", QVariant.DateTime)])
vlayer.updateFields()
tp = vlayer.temporalProperties()
tp.setIsActive(True)
tp.setMode(qgis.core.QgsVectorLayerTemporalProperties.ModeFeatureDateTimeInstantFromField)
tp.setStartField("time")
vlayer.updateFields()

QgsProject.instance().addMapLayer(vlayer)

ts_1 =matrix[:, 0]
points = ts_1[ts_1 != empty_point_wkt]

qgis_fields_list = []
for wkt in np.nditer(points, flags=['refs_ok']):
    feat = QgsFeature(vlayer.fields())
    feat.setAttributes([datetime_obj])  # Set its attributes

    # Create geometry from WKT string
    geom = QgsGeometry.fromWkt(wkt.item())
    feat.setGeometry(geom)  # Set its geometry
    qgis_fields_list.append(feat)

vlayer.startEditing()
vlayer.addFeatures(qgis_fields_list) # Add list of features to vlayer
vlayer.commitChanges()
iface.vectorLayerTools().stopEditing(self.vlayer)


In [3]:
%%time
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from pymeos.db.psycopg import MobilityDB
from datetime import datetime, timedelta 
import numpy as np
from shapely.geometry import Point



host = "localhost"
port = 5432
db = "mobilitydb"
user = "postgres"
password = "postgres"

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

cursor.execute(f"SELECT MMSI FROM public.PyMEOS_demo;")
mmsi_list = cursor.fetchall()

rows = {}
for mmsi in mmsi_list:
    ship_mmsi = mmsi[0]
    cursor.execute(f"SELECT * FROM public.PyMEOS_demo WHERE MMSI = {ship_mmsi} ;")
    _, trajectory, sog = cursor.fetchone()
    rows[mmsi] = trajectory

start_date = datetime(2023, 6, 1, 0, 0, 0)
time_delta = timedelta(minutes=1)
time_ranges = [start_date + i * time_delta for i in range(1440)]

mmsi_index = {}
empty_point_wkt = "POINT EMPTY"  # Use a string for empty point representation
# Create a CuPy matrix of size 1440 x len(mmsi_list) with empty points
matrix = cp.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)

for i in range(len(mmsi_list)):
    try:
        mmsi_index[mmsi_list[i][0]] = i
        traj = rows[mmsi_list[i][0]]
        if traj.num_instants() > 3:
            traj = traj.temporal_precision(timedelta(minutes=1))
            start = traj.start_timestamp()
            traj_resampled = traj.temporal_sample(start=start, duration=timedelta(minutes=1))
            start_time = traj_resampled.start_timestamp().replace(tzinfo=None)
            end_time = traj_resampled.end_timestamp().replace(tzinfo=None)
            start_index = time_ranges.index(start_time)
            end_index = time_ranges.index(end_time)
            values = traj_resampled.values()

            for j in range(start_index, end_index):
                # update the matrix with the point
                matrix[i, j] = values[j - start_index].wkt
    except Exception as e:
        pass

# Convert CuPy matrix back to NumPy if needed
numpy_matrix = cp.asnumpy(matrix)



AttributeError: 'str' object has no attribute 'size'

In [1]:
import cupy as cp

In [2]:
cp.random.randint(0, 100, size = (5000, 1440))
#matrix = cp.full((len(mmsi_list), 1440), empty_point_wkt, dtype=object)

array([[ 8, 79, 45, ..., 57, 47, 65],
       [74, 40,  0, ..., 16, 14, 88],
       [41, 62, 94, ..., 53, 48, 57],
       ...,
       [54, 60, 42, ..., 92,  1, 48],
       [44,  6, 20, ..., 77, 90, 59],
       [ 5, 70, 99, ..., 74, 51, 40]])

In [6]:
import cupy as cp

# Create a large matrix on the GPU using CuPy
matrix_size = (10000, 10000)  # Example large matrix size
matrix = cp.random.rand(*matrix_size).astype(cp.float32)





In [8]:
%%timeit
result_matrix_host = cp.asnumpy(matrix)


45.1 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
dir(cp)

['AxisError',
 'ComplexWarning',
 'DataSource',
 'ElementwiseKernel',
 'Inf',
 'Infinity',
 'ModuleDeprecationWarning',
 'NAN',
 'NINF',
 'NZERO',
 'NaN',
 'PINF',
 'PZERO',
 'RankWarning',
 'RawKernel',
 'RawModule',
 'ReductionKernel',
 'TooHardError',
 'VisibleDeprecationWarning',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__getattr__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_binary',
 '_core',
 '_creation',
 '_cupy',
 '_cupyx',
 '_default_memory_pool',
 '_default_pinned_memory_pool',
 '_deprecated_apis',
 '_embed_signatures',
 '_environment',
 '_functional',
 '_functools',
 '_indexing',
 '_io',
 '_logic',
 '_manipulation',
 '_math',
 '_misc',
 '_numpy',
 '_padding',
 '_sorting',
 '_statistics',
 '_sys',
 '_util',
 '_version',
 'abs',
 'absolute',
 'add',
 'all',
 'allclose',
 'alltrue',
 'amax',
 'amin',
 'angle',
 'any',
 'append',
 'apply_along_axis',
 'arange',
 'arccos',
 'arccosh',
 'arcsin',
 'arcsinh',
 'arctan